# Imports Packages and Set Defaults

In [1]:
from pandas_plink import read_plink
from scipy.special import softmax
import pandas as pd
import numpy as np
from numpy.random import default_rng

In [2]:
rng = default_rng(seed=10)
path = '../data/' # relative path to data directory

# Import Data

In [3]:
bim, fam, bed = read_plink(path + 'CEDAR/CEDAR')

Mapping files: 100%|██████████| 3/3 [00:00<00:00,  6.04it/s]


# Data Exploration and Pre-Processing

In [4]:
bed # [730K SNPs x 322 ppl]

dask.array<transpose, shape=(730525, 322), dtype=float32, chunksize=(1024, 322), chunktype=numpy.ndarray>

In [5]:
X = bed.blocks[5:7].compute().T # [num ppl x num SNPs]

print(X.shape)
print(X)

(322, 2048)
[[1. 2. 2. ... 2. 2. 2.]
 [2. 2. 2. ... 2. 2. 2.]
 [2. 2. 2. ... 2. 2. 2.]
 ...
 [2. 0. 1. ... 2. 1. 2.]
 [2. 2. 2. ... 2. 2. 2.]
 [1. 2. 2. ... 2. 1. 2.]]


#### Check and Remove NaN Values

If a SNP is NaN for any invidual, remove the entire SNP for all individuals in the dataset.

In [6]:
num_nan = np.count_nonzero(np.isnan(X))
print(num_nan, f'NaN values in data')

X_snp_nan = np.isnan(X.sum(axis=0))
num_nan_snp = np.count_nonzero(X_snp_nan)
print(num_nan_snp, f'different SNPs that contain a NaN value')

3345 NaN values in data
819 different SNPs that contain a NaN value


In [7]:
X = X[:, ~np.any(np.isnan(X),axis=0)] # remove SNP columns w/ Nan

print(X.shape)
print(X)

(322, 1229)
[[1. 2. 2. ... 2. 0. 2.]
 [2. 2. 2. ... 2. 2. 2.]
 [2. 2. 2. ... 2. 1. 2.]
 ...
 [2. 0. 1. ... 2. 1. 1.]
 [2. 2. 2. ... 2. 2. 2.]
 [1. 2. 2. ... 2. 1. 1.]]


Binary annotations, start with $1, 5, 20$ annotations. Generate from Bernoulli with $p=0.1$.  For $1, 5$ annotations, set entire weight vector to $1$. For $20$ annotations, set weight vector for $10$ elements to 0 and for the remaining $10$, set them to $1/10$. Pick each group of these 10 vectors randomly from uniform distribution.

# Generate Synthetic Data

In [13]:
# default values
num_casual_snp = 10
num_ppl, num_snp = X.shape # [322 ppl x 1229 SNPs]
num_annotations = 5

In [14]:
# sample functional annotations [num_snp x num_annotations]
A = rng.binomial(n=1, p=0.1, size=(num_snp, num_annotations)) # binomial with n=1 is bernoulli

# generate weight vector w [num_snp x 1]
w = np.ones(num_annotations)

# generate prior causality vector pi [num_snp x 1]
pi = softmax(A @ w)

In [15]:
# sample casual SNPs [num_casual_snp x 1] according to pi
causal_one_hot = rng.multinomial(n=1, pvals=pi, size=num_casual_snp) # multinomial with n=1 is categorical
causal_idx = causal_one_hot.argmax(axis=-1) # convert from one-hot encoding to arr of idicies with causal SNPs

# sample effect size [num_snp x 1]
beta = np.zeros(num_snp)
beta[causal_idx] = rng.multivariate_normal(mean=np.zeros(num_casual_snp), 
                                           cov=np.eye(num_casual_snp))

# sample error [num_ppl x 1]
eps = rng.multivariate_normal(mean=np.zeros(num_ppl), 
                              cov=np.eye(num_ppl) * 0.1)

# generate phenotype [num_ppl,]
y = X @ beta + eps

## Saving Data

In [19]:
np.save(path + 'genotype.npy', X)
np.save(path + 'simulated_phenotype.npy', y)
np.save(path + 'annotation_weight_vector.npy', w)

In [20]:
# binary classification for SNP causality
# 1 indicates SNP is causal, 0 indicates SNP is not
snp_classification = np.zeros(num_snp)
snp_classification[causal_idx] = 1

np.save(path + 'snp_classification.npy', snp_classification)